**Linear Panel Data and Production Technology**

In [27]:
import pandas as pd 
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS, RandomEffects
from scipy.stats import chi2

**1 Cobb-Douglas Production**

In [28]:

# Indlæs data fra firms.csv
dat = pd.read_csv('firms.csv') 

In [29]:
dat.sample(5) # Returnerer en tilfældig prøve på 5 rækker fra dat-dataframen.

,firmid,year,lcap,lemp,ldsa
196,17,1972,1.321425,1.216899,0.978104
1881,157,1977,-1.289410,-1.400780,-1.602180
353,30,1973,-0.995853,-1.110700,-1.430240
2438,204,1970,-2.389200,-3.093220,-2.863850
79,7,1975,1.181523,1.023383,0.749660


In [30]:
dat.year.unique() # returnerer en numpy-array med alle unikke værdier i kolonnen year fra dataframen dat

array([1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979])

In [31]:
print(dat.columns)


Index(['firmid', 'year', 'lcap', 'lemp', 'ldsa'], dtype='object')


In [32]:
# Omdøb variabelnavne for konsistens
expected_cols = {'lemp': 'lempl'}
dat.rename(columns={k: v for k, v in expected_cols.items() if k in dat.columns}, inplace=True)

# Tjek om alle nødvendige variabler er til stede
required_vars = {'ldsa', 'lcap', 'lempl'}
missing_vars = required_vars - set(dat.columns)

if missing_vars:
    raise ValueError(f"Manglende variabler i datasæt: {missing_vars}")

# Konverter til paneldata format
dat['year'] = dat.index % 12  # Antager år er implicit givet i data
dat = dat.set_index(['year', dat.index])

# Estimer Pooled OLS
pooled_ols = smf.ols('ldsa ~ lcap + lempl', data=dat).fit()

# Estimer Fixed Effects (FE)
fe_model = PanelOLS.from_formula('ldsa ~ lcap + lempl + EntityEffects', data=dat).fit()

# Estimer Random Effects (RE)
re_model = RandomEffects.from_formula('ldsa ~ lcap + lempl', data=dat).fit()

# Hypotesetest: Konstant skalaafkast
beta_k, beta_l = fe_model.params['lcap'], fe_model.params['lempl']
wald_stat = ((beta_k + beta_l - 1)**2) / (fe_model.cov.iloc[0, 0] + fe_model.cov.iloc[1, 1])
p_value = 1 - chi2.cdf(wald_stat, 1)

# Hausman-test mellem FE og RE
diff = fe_model.params - re_model.params
var_diff = fe_model.cov + re_model.cov
hausman_stat = diff.T @ np.linalg.inv(var_diff) @ diff
hausman_p_value = 1 - chi2.cdf(hausman_stat, len(diff))

# Udskriv resultater
print("Pooled OLS:\n", pooled_ols.summary())
print("\nFixed Effects:\n", fe_model.summary)
print("\nRandom Effects:\n", re_model.summary)
print(f"\nWald-test for konstant skalaafkast: Test-statistik = {wald_stat:.3f}, p-værdi = {p_value:.3f}")
print(f"\nHausman-test mellem FE og RE: Test-statistik = {hausman_stat:.3f}, p-værdi = {hausman_p_value:.3f}")


Pooled OLS:
                             OLS Regression Results                            
Dep. Variable:                   ldsa   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                 2.807e+04
Date:                Tue, 18 Feb 2025   Prob (F-statistic):               0.00
Time:                        11:14:08   Log-Likelihood:                -2125.9
No. Observations:                5292   AIC:                             4258.
Df Residuals:                    5289   BIC:                             4277.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.536e-08      0.005   3.09